In [20]:
import PIL
from keras import models
from keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import os
from tensorflow.keras import callbacks
import pandas as pd

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

batch_size = 64
epochs = 10

#Train
dataframe = pd.read_csv( '/media/tohn/SSD/ImageForTrainTest/train.csv')
base_dir = '/media/tohn/SSD/ImageForTrainTest/'
os.chdir(base_dir)
train_dir = os.path.join(base_dir, 'train')

#validation
valframe = pd.read_csv( '/media/tohn/SSD/ImageForTrainTest/validation.csv')
validation_dir = os.path.join(base_dir, 'validation')

In [8]:
dataframe

,Unnamed: 0,Case,Abs Position,Sub Position,Class,Sub_class,Path Crop,Views,filename
0,0,40,P3,P31,Normal,Normal,/media/tohn/HDD/VISION_dataset/USAI/croppedPre...,FP-B,P3_Case_040-1.jpg
1,1,194,P3,P31,Normal,Normal,/media/tohn/HDD/VISION_dataset/USAI/US images ...,FP-B,P3.1C194.jpg
2,2,59,P3,P31,Normal,Normal,/media/tohn/HDD/VISION_dataset/USAI/croppedPre...,FP-B,P3_Case_059-1.jpg
3,3,105,P3,P31,Normal,Normal,/media/tohn/HDD/VISION_dataset/USAI/US images ...,FP-B,P3.1C105.jpg
4,4,14,P3,P31,Normal,Normal,/media/tohn/HDD/VISION_dataset/USAI/croppedPre...,FP-B,P3_Case_014-1.jpg
...,...,...,...,...,...,...,...,...,...
2503,2503,51,P7,P72,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-D,AB11 P7-2 C051.JPG
2504,2504,81,P7,P72,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-D,AB11 P7-2 C081.JPG
2505,2505,94,P7,P72,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-D,AB11 P7-2 C094.JPG
2506,2506,72,P7,P72,Abnormal,AB11,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-D,AB11 P7-2 C072.JPG


In [22]:
# array= dataframe['Class'].map({'Normal':0,'Abnormal':1}).values
# # key = list()
# # for i in range(0,2508):
# #     k = i
# #     key.append(k)
# # classs = dict(zip(key, array))    
# classs = pd.DataFrame(array)
# dataframe0= dataframe.join(classs)
# # dataframe0[0]

In [10]:
from efficientnet.keras import EfficientNetB0 as Net
from efficientnet.keras import center_crop_and_resize, preprocess_input
conv_base = Net(weights='imagenet')
height = width = conv_base.input_shape[1]
input_shape = (height, width, 3)

In [25]:
# loading pretrained conv base model
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

# create new model with a new classification layer
x = conv_base.output  
global_average_layer = layers.GlobalAveragePooling2D(name = 'head_pooling')(x)
dropout_layer_1 = layers.Dropout(0.50,name = 'head_dropout')(global_average_layer)
prediction_layer = layers.Dense(2, activation='softmax',name = 'prediction_layer')(dropout_layer_1)

model = models.Model(inputs= conv_base.input, outputs=prediction_layer) 
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='conv_base_b3.png', show_shapes=True)
from IPython.display import Image
Image(filename='conv_base_b3.png')

In [12]:
#showing before&after freezing
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))
#conv_base.trainable = False  # freeze เพื่อรักษา convolutional base's weight
for layer in conv_base.layers:
    layer.trainable = False
print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))  #freez แล้วจะเหลือ max pool and dense

This is the number of trainable layers before freezing the conv base: 213
This is the number of trainable layers after freezing the conv base: 2


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# import preprocess_crop
# from keras.applications.inception_v3 import preprocess_input

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

Found 2508 validated image filenames belonging to 2 classes.


In [15]:
test_generator = test_datagen.flow_from_dataframe(
        dataframe = valframe,
        directory = validation_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

Found 405 validated image filenames belonging to 2 classes.


In [16]:
os.chdir('/media/tohn/SSD/unfreez_model/TrainB0_NA_UnBlock1')

root_logdir = '/media/tohn/SSD/unfreez_model/TrainB0_NA_UnBlock1/my_logs'
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir,run_id)
run_logdir = get_run_logdir()

tensorboard_cb = callbacks.TensorBoard(log_dir = run_logdir)


os.makedirs("./models", exist_ok=True)

def avoid_error(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass

In [17]:
!pwd

/media/tohn/SSD/unfreez_model/TrainB0_NA_UnBlock1


In [18]:
#Training
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      avoid_error(train_generator),
      steps_per_epoch= len(dataframe)//batch_size,
      epochs=epochs,
      validation_data=avoid_error(test_generator), 
      validation_steps= len(valframe) //batch_size,
      use_multiprocessing=True, 
      workers=15,
      max_queue_size=80,
      callbacks = [tensorboard_cb])

model.save('./models/B0_R1.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
39/39 [==============================] - 87s 2s/step - loss: 0.7523 - acc: 0.5152 - val_loss: 0.5929 - val_acc: 0.7161
Epoch 2/10
39/39 [==============================] - 53s 1s/step - loss: 0.7437 - acc: 0.5090 - val_loss: 0.5986 - val_acc: 0.7188
Epoch 3/10
39/39 [==============================] - 53s 1s/step - loss: 0.7349 - acc: 0.5291 - val_loss: 0.5928 - val_acc: 0.7266
Epoch 4/10
39/39 [==============================] - 52s 1s/step - loss: 0.7222 - acc: 0.5368 - val_loss: 0.5812 - val_acc: 0.7448
Epoch 5/10
39/39 [==============================] - 52s 1s/step - loss: 0.7262 - acc: 0.5229 - val_loss: 0.5846 - val_acc: 0.7396
Epoch 6/10
39/39 [==============================] - 52s 1s/step - loss: 0.7151 - acc: 0.5466 - val_loss: 0.5840 - val_acc: 0.7370
Epoch 7/10
39/39 [==============================] - 53s 1s/step - loss: 0.6989 - acc: 0.5700 - val_loss: 0.5761 - val_acc: 0.7578
Epoch 8/10
39/

In [21]:
#Unfreez
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block7a_se_excite':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))  

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

run_logdir = get_run_logdir()

tensorboard_cb = callbacks.TensorBoard(run_logdir)
#early_stop_cb = callbacks.EarlyStopping(monitor='val_acc', patience=66, mode= 'max')

history = model.fit_generator(
      avoid_error(train_generator),
      steps_per_epoch= len(dataframe)//batch_size,
      epochs=epochs,
      validation_data=avoid_error(test_generator), 
      validation_steps= len(valframe) //batch_size,
      use_multiprocessing=True, 
      workers=15,
      max_queue_size=80,
      callbacks = [tensorboard_cb])

model.save('./models/B0_R2_unfreez7.h5')

This is the number of trainable layers after freezing the conv base: 8
Epoch 1/10
39/39 [==============================] - 11s 279ms/step - loss: 0.6194 - acc: 0.6623 - val_loss: 0.5420 - val_acc: 0.8125
Epoch 2/10
39/39 [==============================] - 14s 353ms/step - loss: 0.5453 - acc: 0.7408 - val_loss: 0.4954 - val_acc: 0.8281
Epoch 3/10
39/39 [==============================] - 12s 320ms/step - loss: 0.5009 - acc: 0.7648 - val_loss: 0.4470 - val_acc: 0.9219
Epoch 4/10
39/39 [==============================] - 13s 323ms/step - loss: 0.4663 - acc: 0.7855 - val_loss: 0.4486 - val_acc: 0.8906
Epoch 5/10
39/39 [==============================] - 13s 332ms/step - loss: 0.4923 - acc: 0.7612 - val_loss: 0.3589 - val_acc: 0.9375
Epoch 6/10
39/39 [==============================] - 14s 349ms/step - loss: 0.4858 - acc: 0.7700 - val_loss: 0.3775 - val_acc: 0.9375
Epoch 7/10
39/39 [==============================] - 12s 319ms/step - loss: 0.4285 - acc: 0.8153 - val_loss: 0.3150 - val_acc: 0.952